In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from awpy import Demo
import os
import uuid
from cs_bahtml.radars import RADARS
from cs_bahtml.plotting import position_transform, add_player

In [3]:
dem = Demo(file="..\\..\\demos\\astralis-vs-faze-m1-nuke.dem")

In [17]:
dem.ticks.query("tick == 11378").current_equip_value

113780    4400
113781    4600
113782    4000
113783    3850
113784    5600
113785    5500
113786    5500
113787    4600
113788    4600
113789    4200
Name: current_equip_value, dtype: uint32

In [10]:
dem.ticks

AttributeError: 'str' object has no attribute 'ticks'

In [42]:
(11783-11378)/64

6.328125

In [30]:
dem.grenades.query("thrower == 'device' and entity_id == '121'")

,thrower_steamid,thrower,grenade_type,tick,X,Y,Z,entity_id,round
8239,76561197987713664,device,incendiary_grenade,11783,1643.15625,-1226.09375,-329.28125,121,3
8241,76561197987713664,device,incendiary_grenade,11784,1630.75,-1232.6875,-324.375,121,3
8243,76561197987713664,device,incendiary_grenade,11785,1618.3125,-1239.28125,-319.53125,121,3
8245,76561197987713664,device,incendiary_grenade,11786,1605.90625,-1245.875,-314.8125,121,3
8247,76561197987713664,device,incendiary_grenade,11787,1593.46875,-1252.46875,-310.125,121,3
...,...,...,...,...,...,...,...,...,...
8580,76561197987713664,device,incendiary_grenade,11907,103.1875,-2043.75,-317.1875,121,3
8586,76561197987713664,device,incendiary_grenade,11908,90.78125,-2050.34375,-321.96875,121,3
8592,76561197987713664,device,incendiary_grenade,11909,78.34375,-2056.9375,-326.84375,121,3
8598,76561197987713664,device,incendiary_grenade,11910,65.9375,-2063.53125,-331.78125,121,3


In [31]:
11911-11783

128

In [4]:
map = "de_ancient"
radar = RADARS[map].copy()
radar.show()


In [9]:
radar = RADARS[map].copy()
# Create a draw object
# Define the coordinates of the dot
x, y = -808.5625, -664.84375
player_rad = position_transform(map, 60, "x") - position_transform(map, 0, "x")
radar = add_player(radar, position_transform(map,x,"x"), position_transform(map,y,"y"), player_rad, "t")
# Show the image with the dot
radar.show()

In [5]:
def parse_demo(demo_object,maximum_roundtime = 120,rotate_seconds = 10):
    map_name = demo_object.header.map_name
    tick_rate = 64
    grenades_df = demo_object.grenades
    interesting_grenades = ["smoke","molotov","incendiary_grenade"]
    grenades_df = grenades_df[grenades_df.grenade_type.isin(interesting_grenades)]
    grenades_df.loc[grenades_df['grenade_type'] == 'incendiary_grenade', 'grenade_type'] = 'molotov'
    tick_df = demo_object.ticks
    team_info = tick_df.groupby(["round","side"]).first()['clan'].reset_index()
    round_df = demo_object.rounds
    buy_df = tick_df.merge(round_df['freeze_end'], left_on='tick', right_on='freeze_end', how='left')
    buy_df = buy_df.groupby(['round','side'])['current_equip_value'].mean().reset_index()
    #TODO: Maybe also look at money to gauge whether its a force?
    buy_df['buy_type'] = buy_df.current_equip_value.apply(lambda x: "eco" if x < 1000 else "force" if x < 3000 else "full")

    out_df = grenades_df.groupby(['entity_id',"round"]).first().reset_index()
    out_df['map_name'] = map_name
    out_df = out_df.merge(round_df[["round","freeze_end"]], on='round', how='left')
    out_df['throwSeconds'] = (out_df["tick"] - out_df["freeze_end"])/tick_rate
    out_df = out_df.query("throwSeconds < @maximum_roundtime") 
    
    last_tick_grenade = grenades_df.groupby(['entity_id','round']).last().reset_index()
    out_df = out_df.merge(last_tick_grenade, on=['entity_id','round'], how='left', suffixes=('', '_last_tick'))
    out_df = out_df.merge(tick_df, left_on=['tick','thrower'], right_on=['tick', "name"], how='left', suffixes=('', '_thrower'))
    out_df['throwCords'] = list(zip(out_df['X_thrower'], out_df['Y_thrower'], out_df['Z_thrower']))
    out_df['landCords'] = list(zip(out_df['X_last_tick'], out_df['Y_last_tick'], out_df['Z_last_tick']))
    out_df['throwerSide'] = out_df['side']
    out_df['opponentSide'] = out_df['throwerSide'].apply(lambda x: "TERRORIST" if x == "CT" else "CT")
    
    out_df = out_df.merge(buy_df, on=['round','side'], how='left')
    out_df = out_df.merge(buy_df, left_on=['round','opponentSide'],right_on=['round','side'], how='left', suffixes = ("","_opponentbuy"))
    out_df['throwerBuyType'] = out_df['buy_type']
    out_df['opponentBuyType'] = out_df['buy_type_opponentbuy']
    

    out_df = out_df.merge(team_info, left_on = ["round","opponentSide"], right_on = ["round", "side"], suffixes = ("","_opponent"))
    out_df['throwerTeam'] = out_df['clan']
    out_df['opponentTeam'] = out_df['clan_opponent']
    out_df['throwerName'] = out_df['thrower']
    out_df['rotate_tick'] = out_df['tick']+rotate_seconds*tick_rate

    player_df = out_df.merge(tick_df, left_on=['tick', 'throwerSide'], right_on=['tick', 'side'], how='left', suffixes=('', '_players_thrown'))
    player_df = player_df.query("health>0")
    player_df['position_players_thrown'] = list(zip(player_df['X_players_thrown'], player_df['Y_players_thrown'], player_df['Z_players_thrown']))
    starting_positions = player_df.groupby(['entity_id', 'round'])['position_players_thrown'].agg(list).reset_index(name="starting_positions")
    player_df = player_df.merge(tick_df, left_on=['rotate_tick', 'throwerSide', 'name_players_thrown'], right_on=['tick', 'side', 'name'], how='left', suffixes=('', '_players'))
    player_df['position_players'] = list(zip(player_df['X_players'], player_df['Y_players'], player_df['Z_players']))
    dead_player_df = player_df[player_df['health_players'] == 0].groupby(['entity_id', 'round'])['position_players'].agg(list).reset_index(name='dead_players')
    alive_players_df = player_df[player_df['health_players'] > 0].groupby(['entity_id', 'round'])['position_players'].agg(list).reset_index(name='alive_players')
    out_df = out_df.merge(dead_player_df, on=['entity_id', 'round'], how='left')
    out_df = out_df.merge(alive_players_df, on=['entity_id', 'round'], how='left')
    out_df = out_df.merge(starting_positions, on=['entity_id', 'round'], how='left')
    out_df.dropna(subset=['alive_players'], inplace=True)
    selected_columns = ['map_name', 'entity_id', 'round', 'throwSeconds','grenade_type', 'throwerName', 'throwerTeam', 'throwerBuyType','opponentTeam', 'opponentBuyType', 'throwerSide', 'throwCords', 'landCords', "starting_positions" ,'dead_players', 'alive_players']
    return out_df[selected_columns]

In [32]:
test = parse_demo(dem)
test

,map_name,entity_id,round,throwSeconds,grenade_type,throwerName,throwerTeam,throwerBuyType,opponentTeam,opponentBuyType,throwerSide,throwCords,landCords,starting_positions,dead_players,alive_players
0,de_nuke,65,10,8.71875,smoke,br0,Astralis,full,FaZe Clan,full,CT,"(840.4488525390625, 70.94966125488281, -415.96...","(112.40625, -326.4375, -413.96875)","[(946.9803466796875, -512.65771484375, -96.746...",NaN,"[(675.0065307617188, -654.7633056640625, -206...."
1,de_nuke,65,18,28.328125,smoke,device,Astralis,full,FaZe Clan,full,TERRORIST,"(-1351.2012939453125, -1043.841796875, -370.96...","(1138.6875, -2040.9375, -413.96875)","[(51.53950881958008, -1027.559814453125, -415....",NaN,"[(107.97000122070312, -1016.2108154296875, -41..."
2,de_nuke,66,15,19.140625,molotov,rain,FaZe Clan,full,Astralis,full,CT,"(1167.05517578125, -1296.9703369140625, -415.9...","(1230.15625, -2312.9375, -410.65625)","[(601.146240234375, -984.1541137695312, -415.9...","[(104.07331848144531, 136.4063262939453, -415....","[(596.6456298828125, -1100.789794921875, -365...."
3,de_nuke,71,6,23.515625,molotov,stavn,Astralis,full,FaZe Clan,full,CT,"(861.3890991210938, -1634.232177734375, -415.4...","(1210.71875, -2321.375, -413.375)","[(308.9505615234375, -561.36865234375, -127.96...","[(923.7300415039062, -1557.551025390625, -415....","[(316.55828857421875, -362.82666015625, -127.9..."
4,de_nuke,92,17,19.421875,molotov,device,Astralis,full,FaZe Clan,force,TERRORIST,"(417.908935546875, -1339.753173828125, -415.96...","(879.21875, -1664.59375, -412.96875)","[(544.7725830078125, -818.166015625, -399.9687...","[(619.3079223632812, -812.4690551757812, -415....","[(667.1747436523438, -768.351806640625, -415.9..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,de_nuke,470,8,5.96875,smoke,rain,FaZe Clan,full,Astralis,full,TERRORIST,"(-640.515380859375, -1418.1844482421875, -415....","(442.25, -2129.75, -413.96875)","[(-892.8883666992188, -1128.3846435546875, -41...","[(219.63461303710938, -2124.313720703125, -415...","[(320.7699890136719, -2034.265380859375, -415...."
158,de_nuke,474,10,27.3125,smoke,karrigan,FaZe Clan,full,Astralis,full,TERRORIST,"(-125.35107421875, -1183.811767578125, -110.79...","(963.9375, -446.6875, -125.96875)","[(-125.35107421875, -1183.811767578125, -110.7...",NaN,"[(-370.3466796875, -1632.046142578125, -415.96..."
159,de_nuke,474,21,8.609375,molotov,ropz,FaZe Clan,full,Astralis,force,CT,"(922.6604614257812, 41.81367111206055, -415.96...","(106.46875, -124.5625, -413.46875)","[(919.3548583984375, -1674.396240234375, -415....",NaN,"[(522.0472412109375, -1249.5416259765625, -415..."
160,de_nuke,475,18,6.921875,smoke,device,Astralis,full,FaZe Clan,full,TERRORIST,"(-1351.1507568359375, -1043.4605712890625, -37...","(1136.90625, -2043.34375, -413.96875)","[(-539.0313720703125, -1147.4931640625, -186.5...",NaN,"[(-311.94775390625, -701.6260986328125, -415.9..."


In [59]:
test.query("throwerName == 'device' and entity_id == '121'")

,map_name,entity_id,round,throwSeconds,grenade_type,throwerName,throwerTeam,opponentTeam,throwerSide,throwCords,landCords,starting_positions,dead_players,alive_players
15,de_nuke,121,3,6.328125,molotov,device,Astralis,FaZe Clan,CT,"(1668.0035400390625, -1212.8868408203125, -401...","(53.5, -2070.125, -336.78125)","[(1146.7567138671875, -444.86102294921875, -15...",NaN,"[(455.9765319824219, -891.97998046875, -251.46..."


In [55]:
test['throwSeconds'] = (test["tick"] - test["freeze_end"])/64

In [51]:
(11783-11378)/64

6.328125

In [15]:
patoolib.extract_archive(os.path.join(data_dir,zip), outdir=data_dir)

INFO patool: Extracting ..\..\demos\betboom-dacha-belgrade-2024-south-america-closed-qualifier-imperial-vs-pain-bo3-9EkA64HrPRXCqSd6gmir7L.rar ...
INFO patool: running "C:\Program Files\7-Zip\7z.EXE" x -o..\..\demos -- ..\..\demos\betboom-dacha-belgrade-2024-south-america-closed-qualifier-imperial-vs-pain-bo3-9EkA64HrPRXCqSd6gmir7L.rar
INFO patool:     with input=
INFO patool: ... ..\..\demos\betboom-dacha-belgrade-2024-south-america-closed-qualifier-imperial-vs-pain-bo3-9EkA64HrPRXCqSd6gmir7L.rar extracted to `..\..\demos'.


'..\\..\\demos'

In [6]:
import pandas as pd
import patoolib
data_dir = "..\\..\\demos"
zip_files = [zip_file for zip_file in os.listdir(data_dir) if zip_file.endswith(".rar")]
data_list = []
for zip_file in zip_files:
    if zip_file.endswith(".rar"):
        print(f"Unzipping {zip}")
        patoolib.extract_archive(os.path.join(data_dir,zip_file), outdir=data_dir)
    dem_files = [dem for dem in os.listdir(data_dir) if dem.endswith(".dem")]
    for demo in dem_files:
        path = os.path.join(data_dir,demo)
        print(path)
        try:
            dem = Demo(file=path)
            df = parse_demo(dem,maximum_roundtime = 120,rotate_seconds = 15)
            data_list.append(df)
        except Exception as e:
            print(f"Error: {e}")
        finally:
            os.remove(path)

df = pd.concat(data_list).reset_index(drop=True)

INFO patool: Extracting ..\..\demos\betboom-dacha-belgrade-2024-south-america-closed-qualifier-imperial-vs-pain-bo3-9EkA64HrPRXCqSd6gmir7L.rar ...
INFO patool: running "C:\Program Files\7-Zip\7z.EXE" x -o..\..\demos -- ..\..\demos\betboom-dacha-belgrade-2024-south-america-closed-qualifier-imperial-vs-pain-bo3-9EkA64HrPRXCqSd6gmir7L.rar
INFO patool:     with input=


Unzipping <class 'zip'>


INFO patool: ... ..\..\demos\betboom-dacha-belgrade-2024-south-america-closed-qualifier-imperial-vs-pain-bo3-9EkA64HrPRXCqSd6gmir7L.rar extracted to `..\..\demos'.


..\..\demos\imperial-vs-pain-m1-inferno.dem
her1
her2
..\..\demos\imperial-vs-pain-m2-nuke.dem
her1


INFO patool: Extracting ..\..\demos\betboom-dacha-belgrade-2024-south-america-closed-qualifier-imperial-vs-pain-bo3-CVDSLFvWUP8JR3P40FL6MM.rar ...
INFO patool: running "C:\Program Files\7-Zip\7z.EXE" x -o..\..\demos -- ..\..\demos\betboom-dacha-belgrade-2024-south-america-closed-qualifier-imperial-vs-pain-bo3-CVDSLFvWUP8JR3P40FL6MM.rar
INFO patool:     with input=


...

..\..\demos\vitality-vs-themongolz-nuke.dem
her1


INFO patool: Extracting ..\..\demos\res-regional-series-3-latam-pain-vs-imperial-bo3-eTuuylcqOsfPslUPTOj5GV.rar ...
INFO patool: running "C:\Program Files\7-Zip\7z.EXE" x -o..\..\demos -- ..\..\demos\res-regional-series-3-latam-pain-vs-imperial-bo3-eTuuylcqOsfPslUPTOj5GV.rar
INFO patool:     with input=


her2
Unzipping <class 'zip'>


INFO patool: ... ..\..\demos\res-regional-series-3-latam-pain-vs-imperial-bo3-eTuuylcqOsfPslUPTOj5GV.rar extracted to `..\..\demos'.


..\..\demos\pain-vs-imperial-m1-nuke.dem
her1
her2
..\..\demos\pain-vs-imperial-m2-mirage.dem
her1
her2
..\..\demos\pain-vs-imperial-m3-inferno.dem
her1
her2


In [7]:
df 

,map_name,entity_id,round,throwSeconds,grenade_type,throwerName,throwerTeam,throwerBuyType,opponentTeam,opponentBuyType,throwerSide,throwCords,landCords,starting_positions,dead_players,alive_players
0,de_inferno,68,8,24.890625,molotov,nyezin,paiN,full,Imperial,full,CT,"(1560.4774169921875, 92.0494155883789, 132.03125)","(1335.59375, 304.59375, 130.03125)","[(1560.4774169921875, 92.0494155883789, 132.03...","[(341.71533203125, 1680.1881103515625, 131.921...","[(1476.939208984375, -82.54473876953125, 132.0..."
1,de_inferno,89,15,62.734375,smoke,decenty,Imperial,full,paiN,full,CT,"(1366.7225341796875, 420.350830078125, 129.03125)","(125.9375, 773.8125, 81.5625)","[(1366.7225341796875, 420.350830078125, 129.03...","[(645.6950073242188, 2044.5159912109375, 136.0...","[(2180.668701171875, 2401.213623046875, 187.60..."
2,de_inferno,102,18,84.90625,molotov,biguzera,paiN,full,Imperial,full,TERRORIST,"(479.0244140625, 2016.96875, 136.0312042236328)","(113.1875, 2556.625, 170.15625)","[(1265.96875, 365.9687194824219, 129.525863647...",NaN,"[(876.0958862304688, 61.241310119628906, 204.0..."
3,de_inferno,114,18,61.25,smoke,noway,Imperial,full,paiN,full,CT,"(930.5641479492188, 2731.20068359375, 133.7768...","(739.90625, 2329.8125, 138.03125)","[(1213.080322265625, 169.05128479003906, 206.1...","[(1213.080322265625, 169.05128479003906, 206.1...","[(2553.89013671875, 1329.1119384765625, 160.03..."
4,de_inferno,144,17,15.0625,smoke,nyezin,paiN,full,Imperial,full,TERRORIST,"(872.96875, 43.96869659423828, 135.27679443359...","(1400.125, 1021.1875, 156.34375)","[(872.96875, 43.96869659423828, 135.2767944335...",NaN,"[(1207.16748046875, 565.2262573242188, 122.967..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32182,de_inferno,1520,4,35.46875,smoke,HEN1-,Imperial,force,paiN Gaming,full,CT,"(1508.730712890625, 1042.893798828125, 156.861...","(1230.3125, 504.125, 117.5625)","[(1850.9664306640625, -173.338134765625, 260.0...","[(665.3968505859375, 2720.89306640625, 146.224...","[(864.4584350585938, -346.825439453125, 89.031..."
32183,de_inferno,1528,4,47.78125,smoke,VINI,Imperial,force,paiN Gaming,full,CT,"(682.7188720703125, 3555.591796875, 141.03125)","(229.53125, 2853.21875, 163.53125)","[(963.9238891601562, -647.1146240234375, 97.54...","[(665.3968505859375, 2720.89306640625, 146.224...","[(288.4042053222656, 685.6383056640625, 26.943..."
32184,de_inferno,1531,4,49.5625,molotov,biguzera,paiN Gaming,full,Imperial,force,TERRORIST,"(329.98138427734375, 3074.42431640625, 168.459...","(726.0, 3377.96875, 132.5)","[(641.1748657226562, 2456.225341796875, 189.03...","[(314.4516906738281, 2590.59375, 164.03125)]","[(738.927001953125, 2197.48388671875, 136.0312..."
32185,de_inferno,1533,4,57.875,smoke,kauez,paiN Gaming,full,Imperial,force,TERRORIST,"(755.7681884765625, 2858.639892578125, 139.581...","(1007.46875, 2697.8125, 132.21875)","[(721.9446411132812, 2266.39990234375, 136.031...","[(314.4516906738281, 2590.59375, 164.03125)]","[(152.00033569335938, 1231.624267578125, 94.83..."


In [8]:
df.to_json('../data/parsed_util.json')